In [1]:
# The code was removed by Watson Studio for sharing.

<h1 style="text-align: center">Capstone Project - The Battle of Neighborhoods</h1>

<h2>1. Introduction</h2>

<h3>1.1 Background</h3>
<p>In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are.</p>

<h3>1.2 Problem</h3>
<p>Data required to determine similarity or dissimilarity between neighborhood might include official division of the city, data about arangement of venues inside the neighborhoods, dominant venues, e.g. Hotels, Banks, and of course the location data for neighborhoods and venues. All of these for both cities, New York and Toronto.
Base on that I could try to segment and cluster both city neighborhoods.
There is no certainty, that these data will be sufficient.
</p>

<h3>1.3 Interest</h3>
<p>Similarities or dissimilarities between cities and they neighborhoods could be very useful for companies.</p>
<p>Lets imagine that company ABC has headquarter or a branch in New York, in neighborhood BCD, and is satisfied with the location and surroundings. They are looking for location, to open a branch in Toronto. One of the requirement is to choose the neighborhood, which is very similar to one in New York.</p>
<p>Segmenting and clustring will be done for all neighborhoods in both cities together, based on popular venues in each neighborhood.</p>
<p>Lets assume that company ABC is a SPA, located in neighborhood with many parks or gardens. I will try to find corresponding neighborhood in Toronto which is similar. Technically both neighborhoods should be placed in same cluster.</p>
<p>Here is a list of open questions:</p>
    <ul>
        <li>Is it possible to determine neighborhoods similar to one in New York based on popular venues located in each neighborhood ?
        <li>Are there any patterns how neighborhoods are organized inside a city ?
        <li>How different or similar the cities are ?
    </ul>
</p>

<h2>2. Data acquisition and cleaning</h2>

<h3>2.1 Data sources</h3>
<p>Data sources for this project will be mostly same as I've used during Module 3. of the course.</p>
<p><b>New York</b> has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough, as well as the the latitude and logitude coordinates of each neighborhood.
I will use dataset provided in the course: <a target="_blank" rel="noreferrer" href="https://geo.nyu.edu/catalog/nyu_2451_34572">https://geo.nyu.edu/catalog/nyu_2451_34572</a>. Data file: <a target="_blank" rel="noreferrer" href="https://cocl.us/new_york_dataset">https://cocl.us/new_york_dataset</a>
</p>
<p>For <b>Toronto</b> I will scrape the following Wikipedia page, <a target="_blank" rel="noreferrer" href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">List_of_postal_codes_of_Canada:_</a> to gather data about boroughs and neighborhoods. Wiki page doesn't contain coridates for neighborhoods. I could use geocoder library to gather these data, but becase of the limitations, probably I will use location data from course <a target="_blank" rel="noreferrer" href="https://cocl.us/Geospatial_data">Geospatial data for Toronto neighborhoods</a></p>
<p>For the geograpical coordinate of the cities I can use geocoder library.</p>
<p>To explore the neighborhoods and segment them, I will be utilizing the Foursquare API (https://foursquare.com). For each neighborhood I will be gathering data about venues. Data are in JSON format.</p>



<h3>2.2 Data cleaning</h3>
<p>For New York neighborhoods data are in JSON format. I need to transform them into table with required data.</p>
<p>I need to extract:</p>
<ul>
    <li><b>Borough</b> - name of the city borough, where neighborhood belongs, see properties.borough, e.g. 'Queens'
    <li><b>Neighborhood</b> - name of the neighborhood, see properties.name, e.g. 'Lefrak City'
    <li><b>Latitude</b> - latitude of the neighborhood, see geometry.coordinates second array element, e.g. '40.736074570830795'
    <li><b>Longitude</b> - longitude of the neighborhood, see geometry.coordinates first array element, e.g. '-73.8625247141374'
</ul>
<p>Example for 'Lefrak City'</p>
<pre>
    {'type': 'Feature',
          'id': 'nyu_2451_34572.190',
          'geometry': {'type': 'Point',
           'coordinates': [-73.8625247141374, 40.736074570830795]},
          'geometry_name': 'geom',
          'properties': {'name': 'Lefrak City',
           'stacked': 2,
           'annoline1': 'Lefrak',
           'annoline2': 'City',
           'annoline3': None,
           'annoangle': 0.0,
           'borough': 'Queens',
           'bbox': [-73.8625247141374,
            40.736074570830795,
            -73.8625247141374,
            40.736074570830795]}
    }
</pre>

<p>Foursquare API responses about venues are in JSON format. I need to transform them into table with required data. Request to Foursquare API will be performed for each neighborhood, based on coordinates. This way I can join neighborhood data with venues data.</p>
<p>I will extract:</p>
<ul>
    <li><b>Venue id</b> - id of the venue, see venue.id, e.g. '4baf59e8f964a520a6f93be3', it could be useful to remove duplicates, if range in query is to big
    <li><b>Venue distance</b> - how far the venue is from point used in request (center of neighborhood), see venue.location.distance, e.g. 376, it could be useful to remove duplicates, if range in query is to big
    <li><b>Venue Name</b> - name of the venue, see venue.name, e.g. 'Queens'
    <li><b>Venue Category</b> - name of the venue category, the <b>most</b> important information, will be used for clustering, see venue.categories.name, e.g. 'Yoga Studio'
    <li><b>Venue Latitude</b> - latitude of the venue, see venue.location.lat, e.g. '40.876843690797934'
    <li><b>Venue Longitude</b> - longitude of the venue, see venue.location.lat, e.g. '-73.90620384419528'
</ul>
<p>Probably I can later remove most of data gathered about venues, since clustering will be based on venues categories and how common there are for the selected neighborhood.</p>
<p>Example data about venue from explore API call</p>
<pre>
            {'reasons': {'count': 0,
              'items': [{'summary': 'This spot is popular',
                'type': 'general',
                'reasonName': 'globalInteractionReason'}]},
             'venue': {'id': '4baf59e8f964a520a6f93be3',
              'name': 'Bikram Yoga',
              'location': {'address': '5500 Broadway',
               'crossStreet': '230th Street',
               'lat': 40.876843690797934,
               'lng': -73.90620384419528,
               'labeledLatLngs': [{'label': 'display',
                 'lat': 40.876843690797934,
                 'lng': -73.90620384419528}],
               'distance': 376,
               'postalCode': '10463',
               'cc': 'US',
               'city': 'Bronx',
               'state': 'NY',
               'country': 'United States',
               'formattedAddress': ['5500 Broadway (230th Street)',
                'Bronx, NY 10463',
                'United States']},
              'categories': [{'id': '4bf58dd8d48988d102941735',
                'name': 'Yoga Studio',
                'pluralName': 'Yoga Studios',
                'shortName': 'Yoga Studio',
                'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
                 'suffix': '.png'},
                'primary': True}],
              'photos': {'count': 0, 'groups': []}},
             'referralId': 'e-0-4baf59e8f964a520a6f93be3-1'}
</pre>
<p>For Toronto neighborhoods data are placed on table on wiki. I need to scrap the HTML page and transform them into table with data below.</p>
<p>I need to extract:</p>
<ul>
    <li><b>Postcode</b> - post code for neighborhood, e.g. 'M5H'
    <li><b>Borough</b> - name of the city borough, where neighborhood belongs, e.g. 'Downtown Toronto'
    <li><b>Neighborhood</b> - name of the neighborhood, e.g. 'Adelide'
</ul>
<p>Then I will group gathered data by Postcode, I will remove not assigned post codes and merge such data with cordiates data, by using post code. Rename of column is required.</p>
<p>Here is description of coordinates data for each post code of Toronto. Data are in CSV format so are easy to use.</p>
<ul>
    <li><b>Postal Code</b> - post code for neighborhood, e.g. 'M1B'
    <li><b>Latitude</b> - latitude of the neighborhood, e.g. '43.8066863'
    <li><b>Longitude</b> - longitude of the neighborhood, e.g. '-79.1943534'
</ul>
<p>After join will take place, I could drop 'Postcode' column.<p>
<p>To be able to join data about neighborhoods for both cities, and have a way to distinguish them, I need to add new column <b>City</b> with respectively 'Toronto' and 'New York' values.</p>
<p>After join I will have dataset with following columns</p>
<ul>
    <li><b>City</b>
    <li><b>Borough</b>
    <li><b>Neighborhood</b>
    <li><b>Latitude</b>
    <li><b>Longitude</b>
</ul>


<h3>2.3 Feature selection</h3>
<p>For clustering, I will try to use popular venues categories for each neighborhoods, see 'Venue Category' above, but I'm not sure if this will be sufficent.</p>
<p>I can group results base on Venue Category, calculate how common they are in particular neighborhood surroundings, and perform clustering base on these data.</p>
<p>Using segmentation and clustering, I hope I can determine the similarity or dissimilarirty of both cities and they neighborhoods, and properly name clustered areas located inside the city.</p>




<h3>2.4 Prepare neighborhoods data for both cities</h3>
<p><b>Note:</b> To keep the report cleaner, I've prepared neighborhoods data, together with latitude and longitude values, for both cities on separate notebooks. I've saved these data as CSV files into WDP project, and Github repository</p>
<p>Links to notebooks:</p>
<ul>
    <li><a target="_blank" rel="noreferrer" href="https://github.com/tomkdgun/Coursera_Capstone/blob/master/The%20Battle%20w2%20-%20Data%20prep%20New%20York%20neighborhoods.ipynb">
        The Battle w2 - Data prep New York neighborhoods.ipynb</a>
    <li><a target="_blank" rel="noreferrer" href="https://github.com/tomkdgun/Coursera_Capstone/blob/master/The%20Battle%20w2%20-%20Data%20prep%20Toronto%20neighborhoods.ipynb">
        The Battle w2 - Data prep Toronto neighborhoods.ipynb</a>
</ul>
<p>Links to prepared datasets:</p>
<ul>
    <li><a target="_blank" rel="noreferrer" href="https://github.com/tomkdgun/Coursera_Capstone/blob/master/neighborhood_new_york_geospatial_data.csv">
        neighborhood_new_york_geospatial_data.csv</a>
    <li><a target="_blank" rel="noreferrer" href="https://github.com/tomkdgun/Coursera_Capstone/blob/master/neighborhood_toronto_geospatial_data.csv">
        neighborhood_toronto_geospatial_data.csv</a>
</ul>

<h2>3. Methodology</h2>

<p>Segmenting and clustring will be done for all neighborhoods in both cities together, based on popular venues in each neighborhood. I can group results base on <b>Venue Category</b>, calculate how common they are in particular neighborhood surroundings, and perform clustering base on these data. Additionally to that I will also try do clustering based on venues which are not food and dring related, and check how this will impact the results.</p>
<p>On separate notebooks linked on section above, I've prepared neighborhoods data, together with latitude and longitude values, for both cities.</p>
<ul>
    <li><a target="_blank" rel="noreferrer" href="https://raw.githubusercontent.com/tomkdgun/Coursera_Capstone/master/neighborhood_toronto_geospatial_data.csv">
        neighborhood_toronto_geospatial_data.csv</a>
    <li><a target="_blank" rel="noreferrer" href="https://raw.githubusercontent.com/tomkdgun/Coursera_Capstone/master/neighborhood_new_york_geospatial_data.csv">
        neighborhood_new_york_geospatial_data.csv</a>
</ul>
<p>I will create pandas dataframe for each dataset. To be able to join data about neighborhoods for both cities, and have a way to distinguish them, I need to add new column City with respectively 'Toronto' and 'New York' values.</p>
<p>Then we will use the Foursquare API <b>explore</b>, to get popular places for each neighborhood in both cities. I will be able to create dataset with all venues, together with venue categories.</p>
<p>To be able to perform clustering I need calculate how common particular venue categories are. This requires grouping by city and neighborhood column.</p>
<p>I will use K-means clustering algorithm. Similar neighborhoods of New York and Toronto, should be labeled in the same cluster.</p>

<h3>Join neighborhoods data for New York and Toronto together</h3>

In [17]:
# import required modules
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


To be able to join dataset, I need to read them using URL to Github repo, and covert them to pandas dataframe, or read from Watson Studio project.

In [3]:
toronto_geospatial_data_url = "https://raw.githubusercontent.com/tomkdgun/Coursera_Capstone/master/neighborhood_toronto_geospatial_data.csv"
new_york_geospatial_data_url = "https://raw.githubusercontent.com/tomkdgun/Coursera_Capstone/master/neighborhood_new_york_geospatial_data.csv"

# toronto_geospatial_data = project.get_file("neighborhood_toronto_geospatial_data.csv")
# new_york_geospatial_data =project.get_file("neighborhood_new_york_geospatial_data.csv")

df_new_york_neighborhood = pd.read_csv(new_york_geospatial_data_url);
df_toronto_neighborhood = pd.read_csv(toronto_geospatial_data_url);

print('Shape of df_new_york_neighborhood', df_new_york_neighborhood.shape)
print('Shape of df_toronto_neighborhood', df_toronto_neighborhood.shape)

df_new_york_neighborhood.head(5)

Shape of df_new_york_neighborhood (306, 4)
Shape of df_toronto_neighborhood (103, 5)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [4]:
df_toronto_neighborhood.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Neighborhood column contains unique values for both cities datasets. I can drop PostalCode for Toronto dataset, add <b>City</b> column and join both datasets into one

In [8]:
df_toronto_neighborhood['Neighborhood'].unique().size

103

In [9]:
df_toronto_neighborhood.drop(['PostalCode'], axis=1, inplace=True)
df_toronto_neighborhood.head(2)

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497


In [10]:
df_toronto_neighborhood.insert(0, 'City', 'Toronto')
df_toronto_neighborhood.head(2)

,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Toronto,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497


In [11]:
df_new_york_neighborhood.insert(0, 'City', 'New York')
df_new_york_neighborhood.head(2)

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939


Concat both datasets into one

In [13]:
# concat both datasets into one
both_city_neighborhood = pd.concat([df_new_york_neighborhood, df_toronto_neighborhood], ignore_index=True)
both_city_neighborhood.head(5)

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585


In [14]:
both_city_neighborhood.tail(5)

,City,Borough,Neighborhood,Latitude,Longitude
404,Toronto,York,Weston,43.706876,-79.518188
405,Toronto,Etobicoke,Westmount,43.696319,-79.532242
406,Toronto,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
407,Toronto,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
408,Toronto,Etobicoke,Northwest,43.706748,-79.594054


In [15]:
both_city_neighborhood.shape

(409, 5)

In [16]:
# save in CSV for possible later use
project.save_data(data=both_city_neighborhood.to_csv(index=False),file_name='both_city_neighborhood.csv',overwrite=True)

{'asset_id': '91d24a83-dae3-42a5-bc88-069bc8231a49',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'both_city_neighborhood.csv',
 'message': 'File saved to project storage.'}

<h3>Get popular venues data for each neighborhood in both cities</h3>

#### Define Foursquare Credentials and Version
Next, I'm going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [18]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: NS1HVDRGPSZRRUGMO2M0E0JRUYTCAO050AARVHQRYYY55SHZ
CLIENT_SECRET:JW5BOFI2VWOFKSL1CMAW3WUBIWAWC1P2OSHF5EHFDP2M00ZS


Define funtion to get venues for provided latitudes and longitudes, for each city and neighborhood

In [20]:
def getNearbyVenues(cities, names, latitudes, longitudes, radius=500, limit=200):
    
    venues_list=[]
    for city, name, lat, lng in zip(cities, names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each city neighborhood and create a new dataframe both_city_venues.

In [21]:
# call getNearbyVenues for each Neighborhood
both_city_venues = getNearbyVenues(cities=both_city_neighborhood['City'],
                                   names=both_city_neighborhood['Neighborhood'],
                                   latitudes=both_city_neighborhood['Latitude'],
                                   longitudes=both_city_neighborhood['Longitude'], radius=500, limit=200
                                  )

In [24]:
print('both_city_venues shape', both_city_venues.shape)
both_city_venues.head(4)

both_city_venues shape (12644, 8)


,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,New York,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,New York,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
3,New York,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


In [25]:
# save for later use
project.save_data(data=both_city_venues.to_csv(index=False),file_name='both_city_venues_big_range500.csv',overwrite=True)

{'asset_id': 'f7a7e360-0368-4262-b07a-0ffb5b4ef437',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'both_city_venues_big_range500.csv',
 'message': 'File saved to project storage.'}

#### Let's explore both_city_venues dataframe

Let's check how many venues were returned for each neighborhood

In [38]:
both_city_venues.groupby(['City', 'Neighborhood']).count()

Neighborhood Latitude  \
City     Neighborhood                                                                
New York Allerton                                                               31   
         Annadale                                                                9   
         Arden Heights                                                           4   
         Arlington                                                               6   
         Arrochar                                                               20   
         Arverne                                                                17   
         Astoria                                                               100   
         Astoria Heights                                                        14   
         Auburndale                                                             21   
         Bath Beach                                                             47   
         Battery Park City                                                     100   
         Bay Ridge                                                              85   
         Bay Terrace                                                            52   
         Baychester                                                             22   
         Bayside                                                                72   
         Bayswater                                                               3   
         Bedford Park                                                           38   
         Bedford Stuyvesant                                                     30   
         Beechhurst                                                             18   
         Bellaire                                                               11   
         Belle Harbor                                                           18   
         Bellerose                                                              18   
         Belmont                                                                96   
         Bensonhurst                                                            37   
         Bergen Beach                                                            6   
         Blissville                                                             17   
         Bloomfield                                                              5   
         Boerum Hill                                                            92   
         Borough Park                                                           20   
         Breezy Point                                                            6   
         Briarwood                                                              11   
         Brighton Beach                                                         42   
         Broad Channel                                                           7   
         Broadway Junction                                                      16   
         Bronxdale                                                              17   
         Brooklyn Heights                                                      100   
         Brookville                                                              2   
         Brownsville                                                            20   
         Bulls Head                                                             46   
         Bushwick                                                               76   
         Butler Manor                                                            6   
         Cambria Heights                                                        12   
         Canarsie                                                                6   
         Carnegie Hill                                                         100   
         Carroll Gardens                                                       100   
         Castle Hil

#### Let's find out how many unique categories can be curated from all the returned venues

In [39]:
print('There are {} uniques categories.'.format(len(both_city_venues['Venue Category'].unique())))

There are 463 uniques categories.


####  Let's check what are the most frequent categories

In [40]:
both_city_venues.groupby('Venue Category').count().sort_values('Venue', ascending=False)

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Pizza Place,481,481,481,481,481,481,481
Coffee Shop,455,455,455,455,455,455,455
Italian Restaurant,375,375,375,375,375,375,375
Deli / Bodega,310,310,310,310,310,310,310
Bakery,279,279,279,279,279,279,279
Café,275,275,275,275,275,275,275
Bar,252,252,252,252,252,252,252
Chinese Restaurant,247,247,247,247,247,247,247
Park,231,231,231,231,231,231,231


#### Split dataset on 3 groups, all categories(both_city_venues), food/drink, others

In [41]:
# split dataset on 3 groups, all categories(both_city_venues), food/drink, others
endstrings = ['Café', 'Coffee', 'Pizza', 'Restaurant', 'Bakery', 'Sandwich', 'Bar', 'Pub', 'Gastropub', 'Breakfast', 'Burger', 'Diner', 'Ice Cream Shop', 'Dessert Shop', 'Steakhouse', 'Chicken', 'Burrito', 'Food Truck', 'Bistro', 'Taco', 'Snack', 'Donut Shop', 'Salad Place']
both_city_venues_others = both_city_venues[~both_city_venues['Venue Category'].str.contains('|'.join(endstrings))]
both_city_venues_food_drink = both_city_venues[both_city_venues['Venue Category'].str.contains('|'.join(endstrings))];


In [43]:
project.save_data(data=both_city_venues_others.to_csv(index=False),file_name='both_city_venues_others_big_range500.csv',overwrite=True)
project.save_data(data=both_city_venues_food_drink.to_csv(index=False),file_name='both_city_venues_food_drink_big_range500.csv',overwrite=True)

{'asset_id': '77a7bc87-9e07-46d8-85bb-58cf6066d4c2',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'both_city_venues_food_drink_big_range500.csv',
 'message': 'File saved to project storage.'}

In [45]:
print('both_city_venues shape', both_city_venues.shape)
print('both_city_venues_others shape', both_city_venues_others.shape)
print('both_city_venues_food_drink shape', both_city_venues_food_drink.shape)

both_city_venues shape (12644, 8)
both_city_venues_others shape (5952, 8)
both_city_venues_food_drink shape (6692, 8)


#### Check the most frequent venue category after excluding food and drink related

In [47]:
both_city_venues_others.groupby('Venue Category').count().sort_values('Venue', ascending=False)

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Deli / Bodega,310,310,310,310,310,310,310
Park,231,231,231,231,231,231,231
Grocery Store,204,204,204,204,204,204,204
Pharmacy,201,201,201,201,201,201,201
Gym,155,155,155,155,155,155,155
Bank,154,154,154,154,154,154,154
Hotel,153,153,153,153,153,153,153
Gym / Fitness Center,145,145,145,145,145,145,145
Cosmetics Shop,125,125,125,125,125,125,125


#### Check most frequent food/drink related categories

In [48]:
both_city_venues_food_drink.groupby('Venue Category').count().sort_values('Venue', ascending=False)

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Pizza Place,481,481,481,481,481,481,481
Coffee Shop,455,455,455,455,455,455,455
Italian Restaurant,375,375,375,375,375,375,375
Bakery,279,279,279,279,279,279,279
Café,275,275,275,275,275,275,275
Bar,252,252,252,252,252,252,252
Chinese Restaurant,247,247,247,247,247,247,247
Sandwich Place,223,223,223,223,223,223,223
American Restaurant,202,202,202,202,202,202,202


## Analyze Each Neighborhood

Filter out venues which are other Neighborhood, Neighborhood category

In [49]:
# filter out venues which have Neighborhood category
both_city_venues = both_city_venues[both_city_venues['Venue Category'] != 'Neighborhood']
both_city_venues_others = both_city_venues_others[both_city_venues_others['Venue Category'] != 'Neighborhood']
both_city_venues_food_drink = both_city_venues_food_drink[both_city_venues_food_drink['Venue Category'] != 'Neighborhood']

print('both_city_venues shape', both_city_venues.shape)
print('both_city_venues_others shape', both_city_venues_others.shape)
print('both_city_venues_food_drink shape', both_city_venues_food_drink.shape)

both_city_venues shape (12635, 8)
both_city_venues_others shape (5943, 8)
both_city_venues_food_drink shape (6692, 8)


#### Convert categorical variable (Venue Category) into dummy/indicator variables

In [52]:
# one hot encoding all venues
both_city_venues_onehot = pd.get_dummies(both_city_venues[['Venue Category']], prefix="", prefix_sep="")
col1 =  both_city_venues['City']
col2 =  both_city_venues['Neighborhood']
# insert back City and Neighborhood at front
both_city_venues_onehot.insert(0, 'Neighborhood', col2)
both_city_venues_onehot.insert(0, 'City', col1)

print('both_city_venues_onehot shape', both_city_venues_onehot.shape)
both_city_venues_onehot.head()

both_city_venues_onehot shape (12635, 464)


,City,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Pho

In [53]:
project.save_data(data=both_city_venues_onehot.to_csv(index=False),file_name='both_city_venues_onehot.csv',overwrite=True)

{'asset_id': 'f46734ad-1191-4fa8-9919-72a511fa39c2',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'both_city_venues_onehot.csv',
 'message': 'File saved to project storage.'}

In [54]:
# one hot encoding others venues
both_city_venues_others_onehot = pd.get_dummies(both_city_venues_others[['Venue Category']], prefix="", prefix_sep="")
col1 =  both_city_venues_others['City']
col2 =  both_city_venues_others['Neighborhood']
# insert back City and Neighborhood at front
both_city_venues_others_onehot.insert(0, 'Neighborhood', col2)
both_city_venues_others_onehot.insert(0, 'City', col1)

print('both_city_venues_others_onehot shape', both_city_venues_others_onehot.shape)
both_city_venues_others_onehot.head(5)

both_city_venues_others_onehot shape (5943, 327)


,City,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bank,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Camera Store,Campground,Candy Store,Casino,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Discount Store,Distillery,Dog Run,Drugstore,Dry Cleaner,Duty-free Shop,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm,Farmers Market,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Fountain,Fraternity House,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,General College & University,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Pool,Housing Development,IT Services,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Jazz Club,Jewelry Store,Kids Store,Lake,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Memorial Site,Men's Store,Metro Station,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Poutine Place,Print Shop,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,River,Road,Rock Climbing Spot,Rock Club,Roller Rink,Roof Deck,Scenic Lookout,School,Science Museum,Sculpture Garden,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Shop,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Soup Place,Souvlaki Shop,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Club,Stadium,State / Provincial Park,Stationery Store,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,Surf Spot,Swim School,Tailor Shop,Tanning Salon,Tattoo Parlor,Taxi Stand,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Theat

In [55]:
# one hot encoding food/drink venues
both_city_venues_food_drink_onehot = pd.get_dummies(both_city_venues_food_drink[['Venue Category']], prefix="", prefix_sep="")
col1 =  both_city_venues_food_drink['City']
col2 =  both_city_venues_food_drink['Neighborhood']
# insert back City and Neighborhood at front
both_city_venues_food_drink_onehot.insert(0, 'Neighborhood', col2)
both_city_venues_food_drink_onehot.insert(0, 'City', col1)

print('both_city_venues_food_drink_onehot shape', both_city_venues_food_drink_onehot.shape)
both_city_venues_food_drink_onehot.head(5)

both_city_venues_food_drink_onehot shape (6692, 139)


,City,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Bakery,Bar,Beach Bar,Bed & Breakfast,Beer Bar,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hookah Bar,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Piano Bar,Pizza Place,Polish Restaurant,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tiki Bar,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,New York,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,New York,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,New York,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,New York,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,New York,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
project.save_data(data=both_city_venues_food_drink_onehot.to_csv(index=False),file_name='both_city_venues_food_drink_onehot.csv',overwrite=True)

{'asset_id': '703fd3a8-90df-416f-ac1f-c8fd51d5f9f9',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'both_city_venues_food_drink_onehot.csv',
 'message': 'File saved to project storage.'}

In [57]:
project.save_data(data=both_city_venues_others_onehot.to_csv(index=False),file_name='both_city_venues_others_onehot.csv',overwrite=True)

{'asset_id': 'dd1bb196-a88e-4326-8e42-546780c66658',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'both_city_venues_others_onehot.csv',
 'message': 'File saved to project storage.'}

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [58]:
both_city_venues_grouped = both_city_venues_onehot.groupby(['City', 'Neighborhood']).mean().reset_index()
print('both_city_venues_grouped shape', both_city_venues_grouped.shape)
both_city_venues_grouped.head(5)

both_city_venues_grouped shape (402, 464)


,City,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Pho

#### Do same grouping for others and food/drink related venues

In [59]:
both_city_venues_others_grouped = both_city_venues_others_onehot.groupby(['City', 'Neighborhood']).mean().reset_index()
print('both_city_venues_others_grouped shape', both_city_venues_others_grouped.shape)

both_city_venues_others_grouped shape (401, 327)


In [60]:
both_city_venues_food_drink_grouped = both_city_venues_food_drink_onehot.groupby(['City', 'Neighborhood']).mean().reset_index()
print('both_city_venues_food_drink_grouped shape', both_city_venues_food_drink_grouped.shape)

both_city_venues_food_drink_grouped shape (367, 139)


#### Let's put most common venues for each neighborhood into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = both_city_venues_grouped['City']
neighborhoods_venues_sorted['Neighborhood'] = both_city_venues_grouped['Neighborhood']

neighborhoods_venues_others_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_others_sorted['City'] = both_city_venues_others_grouped['City']
neighborhoods_venues_others_sorted['Neighborhood'] = both_city_venues_others_grouped['Neighborhood']

neighborhoods_venues_food_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_food_sorted['City'] = both_city_venues_food_drink_grouped['City']
neighborhoods_venues_food_sorted['Neighborhood'] = both_city_venues_food_drink_grouped['Neighborhood']

for ind in np.arange(both_city_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(both_city_venues_grouped.iloc[ind, :], num_top_venues)

for ind in np.arange(both_city_venues_others_grouped.shape[0]):
    neighborhoods_venues_others_sorted.iloc[ind, 2:] = return_most_common_venues(both_city_venues_others_grouped.iloc[ind, :], num_top_venues)

for ind in np.arange(both_city_venues_food_drink_grouped.shape[0]):
    neighborhoods_venues_food_sorted.iloc[ind, 2:] = return_most_common_venues(both_city_venues_food_drink_grouped.iloc[ind, :], num_top_venues)


In [73]:
neighborhoods_venues_sorted.head()


,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Allerton,Pizza Place,Spa,Chinese Restaurant,Pharmacy,Deli / Bodega,Supermarket,Dessert Shop,Donut Shop,Gas Station,Bus Station
1,New York,Annadale,Pizza Place,Park,Food,Train Station,Sports Bar,Diner,Restaurant,Pharmacy,Ethiopian Restaurant,Falafel Restaurant
2,New York,Arden Heights,Pharmacy,Pizza Place,Coffee Shop,Bus Stop,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
3,New York,Arlington,Bus Stop,Grocery Store,Boat or Ferry,American Restaurant,Food,Deli / Bodega,Farm,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,New York,Arrochar,Deli / Bodega,Pizza Place,Italian Restaurant,Bus Stop,Supermarket,Athletics & Sports,Middle Eastern Restaurant,Liquor Store,Hotel,Outdoors & Recreation


In [74]:
neighborhoods_venues_others_sorted.head()


,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Allerton,Deli / Bodega,Pharmacy,Spa,Supermarket,Martial Arts Dojo,Electronics Store,Discount Store,Gas Station,Department Store,Bus Station
1,New York,Annadale,Park,Food,Pharmacy,Train Station,Yoga Studio,Factory,Electronics Store,Event Service,Event Space,Exhibit
2,New York,Arden Heights,Pharmacy,Bus Stop,Yoga Studio,Farm,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory
3,New York,Arlington,Bus Stop,Boat or Ferry,Food,Grocery Store,Deli / Bodega,Yoga Studio,Event Space,Exhibit,Eye Doctor,Factory
4,New York,Arrochar,Deli / Bodega,Bus Stop,Liquor Store,Cosmetics Shop,Hotel,Outdoors & Recreation,Supermarket,Athletics & Sports,Bagel Shop,Event Space


In [75]:
neighborhoods_venues_food_sorted.head()

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Allerton,Pizza Place,Chinese Restaurant,Dessert Shop,Fried Chicken Joint,Breakfast Spot,Spanish Restaurant,Fast Food Restaurant,Bakery,Donut Shop,English Restaurant
1,New York,Annadale,Pizza Place,Restaurant,Sports Bar,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Doner Restaurant,Egyptian Restaurant,French Restaurant
2,New York,Arden Heights,Pizza Place,Coffee Shop,Wine Bar,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,English Restaurant,Empanada Restaurant
3,New York,Arlington,American Restaurant,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
4,New York,Arrochar,Pizza Place,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Mediterranean Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,French Restaurant


## Cluster Neighborhoods
Run *K*-means to cluster the neighborhood into 5 clusters.

In [76]:
# set number of clusters
kclusters = 5

grouped_clustering = both_city_venues_grouped.drop(['City', 'Neighborhood'], 1)
grouped_others_clustering = both_city_venues_others_grouped.drop(['City', 'Neighborhood'], 1)
grouped_food_clustering = both_city_venues_food_drink_grouped.drop(['City', 'Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)
kmeans_others = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_others_clustering)
kmeans_food = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_food_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans_others.labels_[0:10])
print(kmeans_food.labels_[0:10])

[3 3 3 3 3 2 2 3 2 3]
[0 0 0 4 4 1 1 1 0 0]
[4 4 4 2 2 2 2 2 2 2]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [77]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_others_sorted.insert(0, 'Cluster Labels', kmeans_others.labels_)
neighborhoods_venues_food_sorted.insert(0, 'Cluster Labels', kmeans_food.labels_)

In [80]:
both_merged = both_city_neighborhood
both_merged.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585


#### Merge neighborhoods_venues_sorted with both_city_neighborhood to add latitude/longitude for each city neighborhood

In [81]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged_all = both_merged.join(neighborhoods_venues_sorted.set_index(['City', 'Neighborhood']), on=['City', 'Neighborhood'])
merged_others = both_merged.join(neighborhoods_venues_others_sorted.set_index(['City', 'Neighborhood']), on=['City', 'Neighborhood'])
merged_food = both_merged.join(neighborhoods_venues_food_sorted.set_index(['City', 'Neighborhood']), on=['City', 'Neighborhood'])

# since it could be none entry in neighborhoods_venues for coresponding city and neighborhood, cleanup rows with NaN
merged_all.dropna(inplace=True)
merged_others.dropna(inplace=True)
merged_food.dropna(inplace=True)

print(merged_all.shape)
print(merged_others.shape)
print(merged_food.shape)

(406, 16)
(405, 16)
(371, 16)


In [82]:
merged_all.head(3)

,City,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Bronx,Wakefield,40.894705,-73.847201,3.0,Laundromat,Ice Cream Shop,Dessert Shop,Donut Shop,Gas Station,Sandwich Place,Caribbean Restaurant,Pharmacy,Food Truck,Ethiopian Restaurant
1,New York,Bronx,Co-op City,40.874294,-73.829939,3.0,Bus Station,Restaurant,Baseball Field,Accessories Store,Discount Store,Fast Food Restaurant,Gift Shop,Park,Grocery Store,Basketball Court
2,New York,Bronx,Eastchester,40.887556,-73.827806,3.0,Caribbean Restaurant,Bus Station,Bus Stop,Metro Station,Deli / Bodega,Diner,Automotive Shop,Chinese Restaurant,Seafood Restaurant,Fast Food Restaurant


In [83]:
merged_all['Cluster Labels'] = merged_all['Cluster Labels'].astype(int)
merged_others['Cluster Labels'] = merged_others['Cluster Labels'].astype(int)
merged_food['Cluster Labels'] = merged_food['Cluster Labels'].astype(int)

In [84]:
merged_all.head(3)

,City,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Bronx,Wakefield,40.894705,-73.847201,3,Laundromat,Ice Cream Shop,Dessert Shop,Donut Shop,Gas Station,Sandwich Place,Caribbean Restaurant,Pharmacy,Food Truck,Ethiopian Restaurant
1,New York,Bronx,Co-op City,40.874294,-73.829939,3,Bus Station,Restaurant,Baseball Field,Accessories Store,Discount Store,Fast Food Restaurant,Gift Shop,Park,Grocery Store,Basketball Court
2,New York,Bronx,Eastchester,40.887556,-73.827806,3,Caribbean Restaurant,Bus Station,Bus Stop,Metro Station,Deli / Bodega,Diner,Automotive Shop,Chinese Restaurant,Seafood Restaurant,Fast Food Restaurant


In [85]:
# save dataset in project
project.save_data(data=merged_all.to_csv(index=False),file_name='merged_all.csv',overwrite=True)
project.save_data(data=merged_others.to_csv(index=False),file_name='merged_others.csv',overwrite=True)
project.save_data(data=merged_food.to_csv(index=False),file_name='merged_food.csv',overwrite=True)

{'asset_id': '952bb8fd-9d24-4e60-a017-ad26c0a57d33',
 'bucket_name': 'courseracapstone-donotdelete-pr-2afewjmpcmomni',
 'file_name': 'merged_food.csv',
 'message': 'File saved to project storage.'}

<h3>Visualize results of clustering</h3>
<p>I need to get coordinates for both cities, New York and Toronto. I will use these for Folium map creation</p>

In [86]:
Toronto_address = 'Toronto, CA'
NY_address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_to_explorer")
toronto_location = geolocator.geocode(Toronto_address)
toronto_latitude = toronto_location.latitude
toronto_longitude = toronto_location.longitude

print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

ny_location = geolocator.geocode(NY_address)
ny_latitude = ny_location.latitude
ny_longitude = ny_location.longitude

print('The geograpical coordinate of New York City are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.
The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Split datasets for both cities

In [88]:
toronto_merged_all = merged_all[merged_all['City'] == 'Toronto']
toronto_merged_others = merged_others[merged_others['City'] == 'Toronto']
toronto_merged_food = merged_food[merged_food['City'] == 'Toronto']

ny_merged_all = merged_all[merged_all['City'] == 'New York']
ny_merged_others = merged_others[merged_others['City'] == 'New York']
ny_merged_food = merged_food[merged_food['City'] == 'New York']


#### Create a folium map

In [89]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_all['Latitude'], toronto_merged_all['Longitude'], toronto_merged_all['Neighborhood'], toronto_merged_all['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Same for New York

In [90]:
# create map for New York
map_clusters = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged_all['Latitude'], ny_merged_all['Longitude'], ny_merged_all['Neighborhood'], ny_merged_all['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [91]:
# create map
map_clusters_others = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_others['Latitude'], toronto_merged_others['Longitude'], toronto_merged_others['Neighborhood'], toronto_merged_others['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_others)
       
map_clusters_others

Same for New York

In [92]:
# create map
map_clusters_others = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged_others['Latitude'], ny_merged_others['Longitude'], ny_merged_others['Neighborhood'], ny_merged_others['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_others)
       
map_clusters_others

In [93]:
# create map
map_clusters_food = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_food['Latitude'], toronto_merged_food['Longitude'], toronto_merged_food['Neighborhood'], toronto_merged_food['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_food)
       
map_clusters_food

Same for New York

In [94]:
# create map
map_clusters_food = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged_food['Latitude'], ny_merged_food['Longitude'], ny_merged_food['Neighborhood'], ny_merged_food['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_food)
       
map_clusters_food

## Examine Clusters

#### Cluster 1

In [106]:
# Parks
merged_all.loc[merged_all['Cluster Labels'] == 0, merged_all.columns[[0] + [2] + list(range(6, merged_all.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,New York,Riverdale,Park,Bus Station,Plaza,Food Truck,Bank,Gym,Home Service,Yoga Studio,Ethiopian Restaurant,Event Service
27,New York,Clason Point,Park,South American Restaurant,Scenic Lookout,Business Service,Boat or Ferry,Bus Stop,Grocery Store,Pool,Yoga Studio,Falafel Restaurant
35,New York,Spuyten Duyvil,Park,Intersection,Bus Line,Bank,Thai Restaurant,Pharmacy,Tennis Stadium,Waste Facility,Scenic Lookout,Factory
192,New York,Somerville,Park,Yoga Studio,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory
203,New York,Todt Hill,Park,Yoga Studio,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory
320,Toronto,"Agincourt North, L'Amoreaux East, Milliken, St...",Asian Restaurant,Park,Playground,Yoga Studio,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
329,Toronto,York Mills West,Park,Convenience Store,Bank,Bar,Field,Event Service,Event Space,Exhibit,Eye Doctor,Factory
331,Toronto,Parkwoods,Fast Food Restaurant,Food & Drink Shop,Park,Yoga Studio,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
336,Toronto,"CFB Toronto, Downsview East",Other Repair Shop,Airport,Park,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory
346,Toronto,East Toronto,Convenience Store,Park,Coffee Shop,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor


##### Most of venues are included in cluster 1, Coffee Shops are very common in this cluster

#### Cluster 2

In [107]:
# Cluster 2 contains hoods with Parks
merged_all.loc[merged_all['Cluster Labels'] == 1, merged_all.columns[[0] + [2] + list(range(6, merged_all.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
397,Toronto,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Yoga Studio,Field,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory
403,Toronto,"Emery, Humberlea",Baseball Field,Yoga Studio,Field,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory


##### Cluster 2 contains hoods with Parks

#### Cluster 3

In [108]:
# Cluster 3 contains only one hood, Playground and Tennis Court are specific for this clustercommon
merged_all.loc[merged_all['Cluster Labels'] == 2, merged_all.columns[[0] + [2] + list(range(6, merged_all.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,New York,Fieldston,River,High School,Bus Station,Playground,Plaza,Yoga Studio,Farm,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
6,New York,Marble Hill,Discount Store,Coffee Shop,Sandwich Place,Yoga Studio,Donut Shop,Diner,Pharmacy,Clothing Store,Bakery,Kids Store
9,New York,Williamsbridge,Nightclub,Caribbean Restaurant,Bar,Soup Place,Yoga Studio,Field,Event Service,Event Space,Exhibit,Eye Doctor
12,New York,City Island,Harbor / Marina,Thrift / Vintage Store,Ice Cream Shop,Seafood Restaurant,Diner,Music Venue,Spanish Restaurant,Smoke Shop,Park,Bar
22,New York,Port Morris,Storage Facility,Latin American Restaurant,Food Truck,Music Venue,Furniture / Home Store,Flower Shop,Spanish Restaurant,Peruvian Restaurant,Donut Shop,Distillery
24,New York,Hunts Point,Farmers Market,Waste Facility,BBQ Joint,Seafood Restaurant,Grocery Store,Gourmet Shop,Pizza Place,Café,Juice Bar,Bank
46,New York,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,Bagel Shop,American Restaurant,Bar,Sushi Restaurant,Pharmacy,Thai Restaurant
49,New York,Greenpoint,Bar,Pizza Place,Coffee Shop,Café,Cocktail Bar,Furniture / Home Store,Bakery,Record Shop,Mexican Restaurant,French Restaurant
51,New York,Brighton Beach,Eastern European Restaurant,Restaurant,Gourmet Shop,Fast Food Restaurant,Mobile Phone Shop,Sushi Restaurant,Bank,Russian Restaurant,Beach,Bakery
52,New York,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Restaurant,Deli / Bodega,Department Store,Diner,Russian Restaurant,Pizza Place


##### Cluster 3 contains only one hood, Playground and Tennis Court are specific for this cluster

In [109]:
# Cluster 4 contains only one hood, Playground and Tennis Court are specific for this cluster
merged_all.loc[merged_all['Cluster Labels'] == 3, merged_all.columns[[0] + [2] + list(range(6, merged_all.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Wakefield,Laundromat,Ice Cream Shop,Dessert Shop,Donut Shop,Gas Station,Sandwich Place,Caribbean Restaurant,Pharmacy,Food Truck,Ethiopian Restaurant
1,New York,Co-op City,Bus Station,Restaurant,Baseball Field,Accessories Store,Discount Store,Fast Food Restaurant,Gift Shop,Park,Grocery Store,Basketball Court
2,New York,Eastchester,Caribbean Restaurant,Bus Station,Bus Stop,Metro Station,Deli / Bodega,Diner,Automotive Shop,Chinese Restaurant,Seafood Restaurant,Fast Food Restaurant
5,New York,Kingsbridge,Pizza Place,Sandwich Place,Supermarket,Discount Store,Latin American Restaurant,Mexican Restaurant,Deli / Bodega,Bar,Pharmacy,Caribbean Restaurant
7,New York,Woodlawn,Playground,Pub,Food & Drink Shop,Pizza Place,Deli / Bodega,Rental Car Location,Moving Target,Bus Station,Supermarket,Bus Stop
8,New York,Norwood,Pizza Place,Park,Deli / Bodega,Pharmacy,Bank,Coffee Shop,Sandwich Place,Athletics & Sports,Mexican Restaurant,Mobile Phone Shop
10,New York,Baychester,Electronics Store,Men's Store,Sandwich Place,Fast Food Restaurant,Donut Shop,Mexican Restaurant,Laundromat,Gym / Fitness Center,Pizza Place,BBQ Joint
11,New York,Pelham Parkway,Italian Restaurant,Pizza Place,Bus Station,Gym / Fitness Center,Mexican Restaurant,Smoke Shop,Sandwich Place,Eye Doctor,Performing Arts Venue,Liquor Store
13,New York,Bedford Park,Diner,Pizza Place,Chinese Restaurant,Deli / Bodega,Supermarket,Convenience Store,Mexican Restaurant,Bus Station,Sandwich Place,Food Truck
14,New York,University Heights,Pizza Place,Bakery,Food,Latin American Restaurant,Supermarket,Shoe Store,Chinese Restaurant,Bank,Pharmacy,Donut Shop


In [110]:
# Cluster 5 contains only one hood, Playground and Tennis Court are specific for this cluster
merged_all.loc[merged_all['Cluster Labels'] == 4, merged_all.columns[[0] + [2] + list(range(6, merged_all.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
212,New York,Oakwood,Women's Store,Bar,Playground,Bus Stop,Field,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
242,New York,Greenridge,Bowling Alley,Pizza Place,Playground,Bus Stop,Pub,Diner,Event Space,Exhibit,Eye Doctor,Fast Food Restaurant
256,New York,Randall Manor,Bus Stop,Bagel Shop,Park,Deli / Bodega,Pizza Place,Playground,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Service
285,New York,Willowbrook,Bus Stop,Chinese Restaurant,Bagel Shop,Pizza Place,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
303,New York,Bayswater,Park,Tennis Court,Playground,Yoga Studio,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
311,Toronto,Scarborough Village,Playground,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
354,Toronto,"Moore Park, Summerhill East",Tennis Court,Playground,Yoga Studio,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor


In [111]:
# Cluster 0 is created for area around Hotels
merged_others.loc[merged_others['Cluster Labels'] == 0, merged_others.columns[[0] + [2] + list(range(6, merged_others.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Wakefield,Pharmacy,Laundromat,Gas Station,Yoga Studio,Farm,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor
1,New York,Co-op City,Baseball Field,Bus Station,Accessories Store,Grocery Store,Pharmacy,Discount Store,Basketball Court,Gift Shop,Park,Eye Doctor
5,New York,Kingsbridge,Discount Store,Supermarket,Deli / Bodega,Pharmacy,Shopping Plaza,Trail,Furniture / Home Store,Bank,Candy Store,Supplement Shop
6,New York,Marble Hill,Discount Store,Yoga Studio,Video Game Store,Gym,Department Store,Pharmacy,Deli / Bodega,Clothing Store,Tennis Stadium,Bank
8,New York,Norwood,Park,Deli / Bodega,Pharmacy,Bank,Liquor Store,Convenience Store,Plaza,Supermarket,Athletics & Sports,Mobile Phone Shop
12,New York,City Island,Harbor / Marina,Thrift / Vintage Store,History Museum,Bank,Pharmacy,Deli / Bodega,Park,Music Venue,Smoke Shop,Farmers Market
14,New York,University Heights,Food,History Museum,Laundromat,Shoe Store,Pharmacy,Deli / Bodega,Bank,Supermarket,Grocery Store,Convenience Store
15,New York,Morris Heights,Grocery Store,IT Services,Cosmetics Shop,Bank,Pharmacy,Playground,Bus Station,Yoga Studio,Factory,Event Service
16,New York,Fordham,Mobile Phone Shop,Shoe Store,Gym / Fitness Center,Sporting Goods Shop,Pharmacy,Bank,Supplement Shop,Electronics Store,Gym,Video Game Store
17,New York,East Tremont,Shoe Store,Supermarket,Bank,Lounge,Discount Store,Mobile Phone Shop,Deli / Bodega,Bus Stop,Flea Market,Cosmetics Shop


In [112]:
# Cluster 1 Pharmacy and Shopping Mail
merged_others.loc[merged_others['Cluster Labels'] == 1, merged_others.columns[[0] + [2] + list(range(6, merged_others.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,New York,Fieldston,Bus Station,River,Plaza,Playground,High School,Yoga Studio,Duty-free Shop,Electronics Store,Event Service,Event Space
4,New York,Riverdale,Bus Station,Park,Gym,Bank,Plaza,Home Service,Farm,Event Service,Event Space,Exhibit
7,New York,Woodlawn,Playground,Food & Drink Shop,Deli / Bodega,Liquor Store,Bus Station,Bus Stop,Supermarket,Moving Target,Laundromat,Train Station
9,New York,Williamsbridge,Nightclub,Soup Place,Food Court,Food & Drink Shop,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory
10,New York,Baychester,Convenience Store,Arcade,Bus Station,Supermarket,Men's Store,Baseball Field,Bank,Gym / Fitness Center,Electronics Store,Pet Store
11,New York,Pelham Parkway,Bus Station,Food,Performing Arts Venue,Metro Station,Liquor Store,Eye Doctor,Smoke Shop,Gym / Fitness Center,Gas Station,Frozen Yogurt Shop
22,New York,Port Morris,Storage Facility,Music Venue,Brewery,Hardware Store,Furniture / Home Store,Flower Shop,Cupcake Shop,Distillery,Farm,Event Space
27,New York,Clason Point,Park,Business Service,Grocery Store,Boat or Ferry,Pool,Scenic Lookout,Bus Stop,Yoga Studio,Event Service,Event Space
35,New York,Spuyten Duyvil,Park,Tennis Stadium,Intersection,Bank,Waste Facility,Scenic Lookout,Pharmacy,Bus Line,Yoga Studio,Exhibit
45,New York,Edenwald,Food,Fish Market,Grocery Store,Gas Station,Bus Station,Yoga Studio,Farm,Event Service,Event Space,Exhibit


In [113]:
# Cluster 2 contains mixed Shops with Park and Gym, different structure
merged_others.loc[merged_others['Cluster Labels'] == 2, merged_others.columns[[0] + [2] + list(range(6, merged_others.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,New York,Somerville,Park,Yoga Studio,Fountain,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm
203,New York,Todt Hill,Park,Yoga Studio,Fountain,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm
378,Toronto,Glencairn,Park,Yoga Studio,Fountain,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm
404,Toronto,Weston,Park,Yoga Studio,Fountain,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm


In [114]:
# Cluster 4 contains Baseball fields with surroundings 
merged_others.loc[merged_others['Cluster Labels'] == 3, merged_others.columns[[0] + [2] + list(range(6, merged_others.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
332,Toronto,Don Mills North,Gym / Fitness Center,Baseball Field,Yoga Studio,Duty-free Shop,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm
338,Toronto,Downsview Central,Home Service,Baseball Field,Farmers Market,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm
397,Toronto,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Yoga Studio,Duty-free Shop,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm,Farmers Market
403,Toronto,"Emery, Humberlea",Baseball Field,Yoga Studio,Duty-free Shop,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm,Farmers Market


In [115]:
# Cluster 5 contains Parks, playground and Zoo
merged_others.loc[merged_others['Cluster Labels'] == 4, merged_others.columns[[0] + [2] + list(range(6, merged_others.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,New York,Eastchester,Bus Station,Bus Stop,Metro Station,Deli / Bodega,Platform,Bowling Alley,Business Service,Automotive Shop,Food & Drink Shop,Fish & Chips Shop
13,New York,Bedford Park,Deli / Bodega,Bus Station,Convenience Store,Supermarket,Park,Dry Cleaner,Health & Beauty Service,Discount Store,Train Station,Pharmacy
23,New York,Longwood,Community Center,Grocery Store,Deli / Bodega,Farm,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory
28,New York,Throgs Neck,Deli / Bodega,Yoga Studio,Dry Cleaner,Electronics Store,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Farm
29,New York,Country Club,Trail,Liquor Store,Playground,Deli / Bodega,Yoga Studio,Farm,Electronics Store,Event Service,Event Space,Exhibit
32,New York,Van Nest,Deli / Bodega,Supermarket,Cosmetics Shop,Bus Station,Yoga Studio,Farmers Market,Event Space,Exhibit,Eye Doctor,Factory
33,New York,Morris Park,Deli / Bodega,Bank,Jewelry Store,Supermarket,Pharmacy,Buffet,Yoga Studio,Event Space,Exhibit,Eye Doctor
34,New York,Belmont,Deli / Bodega,Bank,Grocery Store,Fish Market,Liquor Store,Food & Drink Shop,Electronics Store,Miscellaneous Shop,Shoe Store,Gas Station
39,New York,Edgewater Park,Deli / Bodega,Bookstore,Food & Drink Shop,Park,Liquor Store,Farm,Event Service,Event Space,Exhibit,Eye Doctor
41,New York,Olinville,Supermarket,Food,Health & Beauty Service,Laundromat,Basketball Court,Deli / Bodega,Yoga Studio,Farm,Event Service,Event Space


In [118]:
# Cluster 1 Coffee Shops, Restaurant
merged_food.loc[merged_food['Cluster Labels'] == 0, merged_food.columns[[0] + [2] + list(range(6, merged_food.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
224,New York,Park Hill,Coffee Shop,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
254,New York,Concord,Peruvian Restaurant,Coffee Shop,Wine Bar,English Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
309,Toronto,Woburn,Coffee Shop,Korean Restaurant,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
312,Toronto,"East Birchmount Park, Ionview, Kennedy Park",Coffee Shop,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
340,Toronto,Victoria Village,Portuguese Restaurant,Coffee Shop,Wine Bar,English Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Egyptian Restaurant
346,Toronto,East Toronto,Coffee Shop,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
377,Toronto,"Lawrence Heights, Lawrence Manor",Vietnamese Restaurant,Coffee Shop,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [119]:
# Cluster 2 Quite area with Bakeries and few Restaurant
merged_food.loc[merged_food['Cluster Labels'] == 1, merged_food.columns[[0] + [2] + list(range(6, merged_food.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,New York,Longwood,Latin American Restaurant,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant
25,New York,Morrisania,Fast Food Restaurant,Donut Shop,Pizza Place,Chinese Restaurant,Ice Cream Shop,Sandwich Place,Mexican Restaurant,Latin American Restaurant,Falafel Restaurant,Ethiopian Restaurant
31,New York,Westchester Square,Fast Food Restaurant,Donut Shop,Pizza Place,Diner,Spanish Restaurant,Mexican Restaurant,Chinese Restaurant,Latin American Restaurant,Bar,Sandwich Place
56,New York,East Flatbush,Bakery,Fast Food Restaurant,Caribbean Restaurant,Chinese Restaurant,Gastropub,French Restaurant,Food Truck,Filipino Restaurant,Falafel Restaurant,Wine Bar
72,New York,East New York,Fast Food Restaurant,Food Truck,Chinese Restaurant,Spanish Restaurant,Salon / Barbershop,Asian Restaurant,Pizza Place,Caribbean Restaurant,Filipino Restaurant,Falafel Restaurant
75,New York,Flatlands,Fried Chicken Joint,Caribbean Restaurant,Fast Food Restaurant,Sandwich Place,Asian Restaurant,Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant
91,New York,Bergen Beach,Donut Shop,Wine Bar,English Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
93,New York,Prospect Park South,Caribbean Restaurant,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Donut Shop,Chinese Restaurant,Latin American Restaurant,Food Truck,Mexican Restaurant,Taco Place
141,New York,East Elmhurst,Donut Shop,Ice Cream Shop,Hotel Bar,Coffee Shop,South American Restaurant,Wine Bar,English Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
148,New York,South Ozone Park,Bar,Donut Shop,Sandwich Place,Food Truck,Fast Food Restaurant,Wine Bar,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [120]:
# Cluster 3 Pizza and Fast Food
merged_food.loc[merged_food['Cluster Labels'] == 2, merged_food.columns[[0] + [2] + list(range(6, merged_food.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Wakefield,Dessert Shop,Food Truck,Donut Shop,Sandwich Place,Ice Cream Shop,Caribbean Restaurant,Ethiopian Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
1,New York,Co-op City,Restaurant,Pizza Place,Chinese Restaurant,Ice Cream Shop,Fast Food Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,English Restaurant,Empanada Restaurant
2,New York,Eastchester,Caribbean Restaurant,Diner,Donut Shop,Fast Food Restaurant,Seafood Restaurant,Pizza Place,Chinese Restaurant,Wine Bar,Filipino Restaurant,Falafel Restaurant
4,New York,Riverdale,Food Truck,Wine Bar,English Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Italian Restaurant
5,New York,Kingsbridge,Pizza Place,Sandwich Place,Latin American Restaurant,Mexican Restaurant,Bar,Bakery,Spanish Restaurant,Fried Chicken Joint,Caribbean Restaurant,Donut Shop
6,New York,Marble Hill,Sandwich Place,Coffee Shop,Diner,American Restaurant,Ice Cream Shop,Donut Shop,Steakhouse,Pizza Place,Bakery,Seafood Restaurant
7,New York,Woodlawn,Pub,Pizza Place,Indian Restaurant,Food Truck,Bar,Italian Restaurant,American Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
10,New York,Baychester,Fried Chicken Joint,American Restaurant,Donut Shop,Breakfast Spot,Sandwich Place,Pizza Place,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Falafel Restaurant
11,New York,Pelham Parkway,Italian Restaurant,Pizza Place,Sandwich Place,Chinese Restaurant,Donut Shop,Ice Cream Shop,Sushi Restaurant,Bakery,Mexican Restaurant,Coffee Shop
12,New York,City Island,Seafood Restaurant,Ice Cream Shop,Italian Restaurant,Pizza Place,Diner,Spanish Restaurant,Bar,American Restaurant,French Restaurant,Eastern European Restaurant


In [121]:
# Cluster 4 Food tracks
merged_food.loc[merged_food['Cluster Labels'] == 3, merged_food.columns[[0] + [2] + list(range(6, merged_food.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,New York,Williamsbridge,Caribbean Restaurant,Bar,Wine Bar,Gastropub,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
207,New York,Port Ivory,Bar,Wine Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Gastropub
212,New York,Oakwood,Bar,Wine Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Gastropub
275,New York,Stuyvesant Town,Bar,Coffee Shop,Cocktail Bar,Wine Bar,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
307,Toronto,"Highland Creek, Rouge Hill, Port Union",Bar,Wine Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Gastropub
329,Toronto,York Mills West,Bar,Wine Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Gastropub
374,Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",Bar,Wine Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Gastropub
408,Toronto,Northwest,Bar,Wine Bar,Ethiopian Restaurant,French Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant,Gastropub


In [122]:
# Cluster 5 Different structure of places
merged_food.loc[merged_food['Cluster Labels'] == 4, merged_food.columns[[0] + [2] + list(range(6, merged_food.shape[1]))]]

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,New York,Norwood,Pizza Place,Sandwich Place,Chinese Restaurant,Restaurant,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Caribbean Restaurant,Bakery,Spanish Restaurant
17,New York,East Tremont,Pizza Place,Café,Breakfast Spot,Mexican Restaurant,Paella Restaurant,Fast Food Restaurant,Restaurant,Egyptian Restaurant,Empanada Restaurant,French Restaurant
20,New York,Melrose,Pizza Place,Diner,Donut Shop,Sandwich Place,Mexican Restaurant,Chinese Restaurant,Ethiopian Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant
21,New York,Mott Haven,Pizza Place,Spanish Restaurant,Latin American Restaurant,Peruvian Restaurant,Café,Burger Joint,Donut Shop,Bakery,Egyptian Restaurant,Empanada Restaurant
32,New York,Van Nest,Pizza Place,Food Truck,Donut Shop,Coffee Shop,Hookah Bar,Wine Bar,English Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
40,New York,Castle Hill,Diner,Spanish Restaurant,Pizza Place,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,French Restaurant,Empanada Restaurant
45,New York,Edenwald,Pizza Place,Chinese Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,English Restaurant,Wine Bar,Fried Chicken Joint
55,New York,Crown Heights,Pizza Place,Café,Sushi Restaurant,Bakery,Salon / Barbershop,Burger Joint,Coffee Shop,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant
78,New York,Coney Island,Caribbean Restaurant,Pizza Place,Café,Wine Bar,Ethiopian Restaurant,Food Truck,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,English Restaurant
80,New York,Borough Park,Pizza Place,Bakery,Fast Food Restaurant,Coffee Shop,Café,Restaurant,American Restaurant,Egyptian Restaurant,Eastern European Restaurant,Empanada Restaurant
